### First Practice Activity of a Recommendation System

To begin, my objective is learn about RecSystems with Reinforcement Learning, but as i don't know anything about RecSys or RL, i decided to understanding each term separately and, after that, finally study the two as a unique theme. Therefore, i will start with Recommendation System, due to the fact of be more simple than RL, beginning with all important definitions in these area of study.

#### First Ideas
The basis of the recommendation system is "What do you want to recommend, for whom, and how?" you can find many ways to make recommendations.

For example, if you have a client who watches a horror movie, you have two main ways to choose a recommendation for him. The first is to recommend other horror movies like the first one, i.e., similar items(films) to the original article in the interaction—that is an item-focused approach.

In other cases, you wish to recommend movies watched by other users similar to the client who watches that horror movie; in that case, you use a User-focused approach.

You can apply these approaches to different types of recommendation systems.


Despite that, you have two ways to make a recommendation system: collaborative and content.

##### Collaborative

Collaborative Methods use past interactions between users and items to make new recommendations using a "User-Item Interactions Matrix." In the middle of the recommendation, it uses the proximity between the users/items to predict. Is it possible to use a model-based approach or a metric to calculate the similarity and just use this metric.

It has some issues, like problems with new Users and Items, but with other methods specific to further information that can surpass this problem.

A lot of methods using a collaborative approach, such as:

        - Memory Based
        - Model-Based

##### Content

Content methods use additional information about users and/or items. In the movie example, you can have information about the Release Year, the Director, the Principal Actors, and many other data.

These methods have much more information and don't suffer with new Users or Items too much because they 

### Practice

I will use the most simple recommendation system for this first recommendation system, a memory-based collaborative model. These recommendation systems use just similarity metrics to make a recommendation, i.e., they don't have any ML model in the middle of the process as the simplest method is the most interesting way to new students like me.

One of the problems of that method is the bad scalability with the new users and new itens, because of the pivot table.

How i gonna use a collaborative approach, i need just the information about the interactions user-item.

I will use three datasets to test the recommendation:
        --- IMDB Movies Dataset  - Reviews
        --- Amazon Books Dataset - Reviews
        --- Steam Games Dataset  - Hours Played

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix
from distanceRecommender import DistanceCollaborativeRecommender

### IMDB MOVIES

In [2]:
names = pd.read_csv("../Datasets/Movies IMDB/info.csv")

names.info()

interactions = pd.read_csv("../Datasets/Movies IMDB/ratings.csv")

interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB


In [3]:
del interactions["timestamp"]

del names["genres"]

# Removing irrelevant information

Merging the two dataframes in one

In [4]:
data = pd.merge(names, interactions)

del names, interactions

data.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),3,4.0
1,1,Toy Story (1995),6,5.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),10,4.0
4,1,Toy Story (1995),11,4.5


In [5]:
users_to_validate = data[data["userId"].isin([357, 22006, 3400])]

Taking a sample of 10 Million to reduce the dimensionality of the DataFrame

Removing users with low count of reviews


Creating a pivot table with the format: 
    
   - Rows: Users
   - Columns: Items

With this pivot we can make the recommendations.

In [6]:
# Code to create the pivot table in a efficient way in csr matrix

def sparse_pivot_table(data, userId_column, itemId_column, ratings_column):
    user_ids = data[userId_column].unique()
    title_ids = data[itemId_column].unique()

    user_to_row = {user_id: i for i, user_id in enumerate(data[userId_column].unique())} # dict used to userNames
    title_to_col = {title: j for j, title in enumerate(data[itemId_column].unique())} # dict used to itemNames

    rows = [user_to_row[user_id] for user_id in data[userId_column]]
    cols = [title_to_col[title] for title in data[itemId_column]]
    ratings = data[ratings_column].tolist()

    sparse_matrix = csr_matrix((ratings, (rows, cols)), shape=(len(user_ids), len(title_ids)))

    user_list = pd.Series(list(user_to_row.keys()))
    item_list = pd.Series(list(title_to_col.keys()))

    return user_list, item_list, sparse_matrix

In [7]:
user_names, movie_names, sparse_matrix = sparse_pivot_table(data, "userId", "title", "rating")

In [8]:
pivot_csr = sparse_matrix

pivot_csc = sparse_matrix.tocsc()

# Creating two sparses tables to make a more efficient code

In [9]:
del data, sparse_matrix

### Using the Collaborative Recommender

In first place i build the recommender in the jupyter, but to make more consistent and clean i build a class DistanceCollaborativeRecommender to make this job.

First, i use a item recommender, you give a item and returns similar items.
After that, i use the Collab. Recommender for item and user, with two differents users.

In [10]:
# function i use to dont repeat code

def get_and_display_recommendation(recommender, user_id):
    user, recommendation, values = recommender.get_recommendation(user_id, need_numeric=True)

    print("Recommendation for the user", user, ": ")
    display(pd.Series(recommendation))

    print("Items of the user", user, ": ")
    display(recommender.get_user_items(user)[:10])

recommender_user_based = DistanceCollaborativeRecommender(pivot_csr, itemNames=movie_names, userNames=user_names,
                                       type_of_recommendation="User", metric="cosine")

recommender_item_based = DistanceCollaborativeRecommender(pivot_csc, itemNames=movie_names, userNames=user_names,
                                       type_of_recommendation="Item", metric="cosine")

#### Similars Items

In [11]:
item, similars = recommender_item_based.get_items_similars("Star Wars")

display(similars)

array([0.5772946 , 0.33464377, 0.27060124, ..., 0.00349453, 0.00399375,
       0.        ])

#### Item-Based

In [12]:
get_and_display_recommendation(recommender_item_based, 357)

Recommendation for the user 357 : 


1212                               Terminator, The (1984)
1184    Star Wars: Episode VI - Return of the Jedi (1983)
1017                                      Die Hard (1988)
537                                   Blade Runner (1982)
1356                                          Jaws (1975)
2830                                  Total Recall (1990)
2630           Ghostbusters (a.k.a. Ghost Busters) (1984)
2486                                   Matrix, The (1999)
2031          Indiana Jones and the Temple of Doom (1984)
2899                                       RoboCop (1987)
dtype: object

Items of the user 357 : 


2555                                      Superman (1978)
2556                                   Superman II (1980)
1175                                        Aliens (1986)
1275            Butch Cassidy and the Sundance Kid (1969)
1173    Raiders of the Lost Ark (Indiana Jones and the...
1171    Star Wars: Episode V - The Empire Strikes Back...
1344           Star Trek III: The Search for Spock (1984)
1345                 Star Trek IV: The Voyage Home (1986)
3611                                       Mad Max (1979)
1052           Willy Wonka & the Chocolate Factory (1971)
dtype: object

In [13]:
get_and_display_recommendation(recommender_item_based, 22006)

Recommendation for the user 22006 : 


2486                                   Matrix, The (1999)
1528                     Men in Black (a.k.a. MIB) (1997)
1242                            Back to the Future (1985)
1171    Star Wars: Episode V - The Empire Strikes Back...
2676                              Sixth Sense, The (1999)
5252                                    Spider-Man (2002)
1212                               Terminator, The (1984)
3487                                     Gladiator (2000)
1263            Indiana Jones and the Last Crusade (1989)
1173    Raiders of the Lost Ark (Indiana Jones and the...
dtype: object

Items of the user 22006 : 


14568                                        Avatar (2009)
4054                                       Hannibal (2001)
2994                                  Sleepy Hollow (1999)
1291                        Alien³ (a.k.a. Alien 3) (1992)
1184     Star Wars: Episode VI - Return of the Jedi (1983)
843                                  Godfather, The (1972)
602                                           Fargo (1996)
586                                          Batman (1989)
3933                     O Brother, Where Art Thou? (2000)
583                      Terminator 2: Judgment Day (1991)
dtype: object

##### Consideration about cosine similarity

A important information is cosine similarity dont considers magnitude, i.e. if a user rate all films they see with 0, is perfectly similar to another user who rates all the same films as 5, is important considers the rating of a film, but the users are similars in some way, when you focus in watched movies.

Cosine Similarity can be a bit dangerous, but if you know the risks, you can consider using this metric.


#### User-Based


In [14]:
get_and_display_recommendation(recommender_user_based, 357)

Recommendation for the user 357 : 


2486                                   Matrix, The (1999)
293                                   Pulp Fiction (1994)
1184    Star Wars: Episode VI - Return of the Jedi (1983)
587                      Silence of the Lambs, The (1991)
315                      Shawshank Redemption, The (1994)
352                                   Forrest Gump (1994)
476                                  Jurassic Park (1993)
108                                     Braveheart (1995)
523                               Schindler's List (1993)
843                                 Godfather, The (1972)
dtype: object

Items of the user 357 : 


2555                                      Superman (1978)
2556                                   Superman II (1980)
1175                                        Aliens (1986)
1275            Butch Cassidy and the Sundance Kid (1969)
1173    Raiders of the Lost Ark (Indiana Jones and the...
1171    Star Wars: Episode V - The Empire Strikes Back...
1344           Star Trek III: The Search for Spock (1984)
1345                 Star Trek IV: The Voyage Home (1986)
3611                                       Mad Max (1979)
1052           Willy Wonka & the Chocolate Factory (1971)
dtype: object

In [15]:
get_and_display_recommendation(recommender_user_based, 22006)

Recommendation for the user 22006 : 


293                                   Pulp Fiction (1994)
315                      Shawshank Redemption, The (1994)
352                                   Forrest Gump (1994)
587                      Silence of the Lambs, The (1991)
257             Star Wars: Episode IV - A New Hope (1977)
2486                                   Matrix, The (1999)
476                                  Jurassic Park (1993)
1171    Star Wars: Episode V - The Empire Strikes Back...
523                               Schindler's List (1993)
49                             Usual Suspects, The (1995)
dtype: object

Items of the user 22006 : 


14568                                        Avatar (2009)
4054                                       Hannibal (2001)
2994                                  Sleepy Hollow (1999)
1291                        Alien³ (a.k.a. Alien 3) (1992)
1184     Star Wars: Episode VI - Return of the Jedi (1983)
843                                  Godfather, The (1972)
602                                           Fargo (1996)
586                                          Batman (1989)
3933                     O Brother, Where Art Thou? (2000)
583                      Terminator 2: Judgment Day (1991)
dtype: object

In [16]:
pivot_csc.shape
# OLD - 2084 Movies and 138300 Users
# NEW - 26729 Movies and 138493 Users
# OBS = Cosine and pearson with same result for users

(138493, 26729)

In [17]:
del pivot_csc, pivot_csr, users_to_validate, user_names, movie_names

### AMAZON BOOKS

The amazon books dataset is a well formatted dataset, with a lot of informations i remove previously, maintaining just the essential to recommend in this notebook. I also perform multiple tests to be safe the pivot table are correct.

In [18]:
ratings = pd.read_csv("../Datasets/Amazon Books/simple_ratings.csv")

ratings.head()

,Title,Id,User_id,review/score
0,Its Only Art If Its Well Hung!,1882931173,AVCGYZL8FQQTD,4.0
1,Dr. Seuss: American Icon,0826414346,A30TK6U7DNS82R,5.0
2,Dr. Seuss: American Icon,0826414346,A3UH4UZ4RSVO82,5.0
3,Dr. Seuss: American Icon,0826414346,A2MVUWT453QH61,4.0
4,Dr. Seuss: American Icon,0826414346,A22X4XUPKF66MR,4.0


In [19]:
ratings.dropna(axis = 0, inplace = True)

ratings.info()

data = ratings.copy()

<class 'pandas.core.frame.DataFrame'>
Index: 461962 entries, 0 to 574999
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Title         461962 non-null  object 
 1   Id            461962 non-null  object 
 2   User_id       461962 non-null  object 
 3   review/score  461962 non-null  float64
dtypes: float64(1), object(3)
memory usage: 17.6+ MB


In [20]:
user_names, item_names, sparse_matrix = sparse_pivot_table(data, "User_id", "Title", "review/score")

In [21]:
recommender_item_based = DistanceCollaborativeRecommender(sparse_matrix.tocsc(), itemNames = item_names, userNames = user_names, 
                               type_of_recommendation="Item", metric = "cosine")

recommender_user_based = DistanceCollaborativeRecommender(sparse_matrix, itemNames = item_names, userNames = user_names, 
                               type_of_recommendation="User", metric = "cosine")

In [22]:
get_and_display_recommendation(recommender_item_based, "A3B25NK1NTDS4M")

Recommendation for the user A3B25NK1NTDS4M : 


26265                                 Bachelors Get Lonely
27704                        Spill the Jackpot ("Mapback")
1993                       The Case of the Haunted Husband
24899                                      Top of the Heap
28395                                  The Battle of Kursk
8796     El Caso De LA Heredera Solitaria/the Case of t...
7915     El Caso Del Patito Que Se Ahogaba/the Case of ...
32798                        The Case of the Backward Mule
6240                           Give 'em The Ax (Dell D213)
24983    Nine men;: A political history of the Supreme ...
dtype: object

Items of the user A3B25NK1NTDS4M : 


30080                                          Red Harvest
30591                                    A Hell of a Woman
30174    A comparison of the "Origin of movement" serie...
35727                                   Where Serpents Lie
26026    The Case of the Howling Dog (Perry Mason Myste...
6621                                ONE FEARFUL YELLOW EYE
5867                                   Muscle for the Wing
dtype: object

In [23]:
get_and_display_recommendation(recommender_user_based, "A3B25NK1NTDS4M")

Recommendation for the user A3B25NK1NTDS4M : 


6741         Farewell, my lovely
11968               The Thin Man
12686              The Glass Key
34539                  Big Sleep
34729         Farewell My Lovely
10861        Farewell, My Lovely
30693             The Dain Curse
30393    Great Gatsby (Everyman)
19225           The Great Gatsby
1224              Fahrenheit 451
dtype: object

Items of the user A3B25NK1NTDS4M : 


30080                                          Red Harvest
30591                                    A Hell of a Woman
30174    A comparison of the "Origin of movement" serie...
35727                                   Where Serpents Lie
26026    The Case of the Howling Dog (Perry Mason Myste...
6621                                ONE FEARFUL YELLOW EYE
5867                                   Muscle for the Wing
dtype: object

In [24]:
del ratings, user_names, item_names, sparse_matrix, data


### STEAM GAMES

Steam Games dataset dont has review information, but has hours played information, so i use the hours instead of reviews in the pivot table.

In [25]:
transactions =  pd.read_csv("../Datasets/Games/new_transactions.csv")

transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   User_ID    200000 non-null  int64  
 1   Game_Name  200000 non-null  object 
 2   Action     200000 non-null  object 
 3   Hours      200000 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB


In [26]:
transactions.head()

,User_ID,Game_Name,Action,Hours
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


In [27]:
games_hours = transactions.drop(axis = 0, index = transactions[transactions["Action"] == "purchase"].index)[["User_ID", "Game_Name", "Hours"]]

first_cut =  games_hours[["Game_Name", "Hours"]].groupby(["Game_Name"]).quantile(0.3)
median_ =    games_hours[["Game_Name", "Hours"]].groupby(["Game_Name"]).quantile(0.5)
third_cut =  games_hours[["Game_Name", "Hours"]].groupby(["Game_Name"]).quantile(0.65)
fourth_cut = games_hours[["Game_Name", "Hours"]].groupby(["Game_Name"]).quantile(0.90)

In [28]:
games_hours["HLow_Played"] = 0
games_hours["HLow_Played"] = games_hours.apply(
                                        lambda x: 1
                                        if 0 < x["Hours"] > 0
                                        else 0,
                                        axis = 1)

games_hours["Low_Played"] = 0
games_hours["Low_Played"] = games_hours.apply(
                                        lambda x: 1
                                        if 0 < x["Hours"] >= first_cut.loc[x["Game_Name"]].iloc[0]
                                        else 0,
                                        axis = 1)

games_hours["Mid_Played"] = 0
games_hours["Mid_Played"] = games_hours.apply(
                                        lambda x: 1
                                        if 0 < x["Hours"] >= median_.loc[x["Game_Name"]].iloc[0]
                                        else 0,
                                        axis = 1)

games_hours["High_Played"] = 0
games_hours["High_Played"] = games_hours.apply(
                                        lambda x: 1
                                        if x["Hours"] >= third_cut.loc[x["Game_Name"]].iloc[0]
                                        else 0,
                                        axis = 1)

games_hours["Extr_Played"] = 0
games_hours["Extr_Played"] = games_hours.apply(
                                        lambda x: 1
                                        if x["Hours"] >= fourth_cut.loc[x["Game_Name"]].iloc[0]
                                        else 0,
                                        axis = 1)

games_hours["Played"] = (games_hours["HLow_Played"] + games_hours["Low_Played"] + 
                         games_hours["Mid_Played"] + games_hours["High_Played"] + games_hours["Extr_Played"])

games_hours.drop(axis = 1, columns = ["HLow_Played", "Low_Played", "Mid_Played", "High_Played", "Extr_Played"], inplace = True)

In [29]:
data = games_hours.copy()

data["User_ID"] = data["User_ID"].astype(str)

user_names, item_names, sparse_matrix = sparse_pivot_table(data, "User_ID", "Game_Name", "Played")

In [30]:
recommender_item_based = DistanceCollaborativeRecommender(sparse_matrix.tocsc(), itemNames = item_names, userNames = user_names,   
                                                          type_of_recommendation="Item", metric = "cosine")
recommender_user_based = DistanceCollaborativeRecommender(sparse_matrix, itemNames = item_names, userNames = user_names, 
                                                          type_of_recommendation="User", metric = "cosine")

In [31]:
get_and_display_recommendation(recommender_item_based, "161896440")

Recommendation for the user 161896440 : 


108                       Dishonored
0         The Elder Scrolls V Skyrim
154                    Borderlands 2
62                          Portal 2
59                      Just Cause 2
10                       Tomb Raider
64     Middle-earth Shadow of Mordor
63          Deus Ex Human Revolution
4                      Left 4 Dead 2
3                  Fallout New Vegas
dtype: object

Items of the user 161896440 : 


848                 Metro 2033 Redux
497    Call of Duty Advanced Warfare
52                         Far Cry 3
13                 BioShock Infinite
22                          BioShock
46                Grand Theft Auto V
226           Metro Last Light Redux
113                 Metro Last Light
991             Batman Arkham Knight
796            Football Manager 2015
dtype: object

In [32]:
get_and_display_recommendation(recommender_user_based, "161896440")

Recommendation for the user 161896440 : 


9                      Team Fortress 2
21                              Dota 2
0           The Elder Scrolls V Skyrim
39     Counter-Strike Global Offensive
4                        Left 4 Dead 2
154                      Borderlands 2
62                            Portal 2
24                         Garry's Mod
280                           PAYDAY 2
120                             Portal
dtype: object

Items of the user 161896440 : 


848                 Metro 2033 Redux
497    Call of Duty Advanced Warfare
52                         Far Cry 3
13                 BioShock Infinite
22                          BioShock
46                Grand Theft Auto V
226           Metro Last Light Redux
113                 Metro Last Light
991             Batman Arkham Knight
796            Football Manager 2015
dtype: object

### OK
#### tests for other datasets - Focus - depois separa ipynbs para cada dataset
##### Amazon Books, Anime and Steam Datasets